# New Unseen Data Prediction Regression

In [2]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor

In [3]:
new_test_data = pd.DataFrame({
    'Vehicle Class': ['Luxury SUV', 'Luxury Car'],
    'Coverage': ['Premium', 'Premium'],
    'Renew Offer Type': ['Offer3', 'Offer1'],
    'EmploymentStatus': ['Retired', 'Employed'],
    'Marital Status': ['Divorced', 'Single'],
    'Education': ['Master', 'College'],
    'Number of Policies': [5.0, 2.0],
    'Monthly Premium Auto': [10.0, 320.0],
    'Total Claim Amount': [10_000.0, 2_000.0],
    'Income': [50_000.00, 13_000.0]
})

new_test_data

,Vehicle Class,Coverage,Renew Offer Type,EmploymentStatus,Marital Status,Education,Number of Policies,Monthly Premium Auto,Total Claim Amount,Income
0,Luxury SUV,Premium,Offer3,Retired,Divorced,Master,5.0,10.0,10000.0,50000.0
1,Luxury Car,Premium,Offer1,Employed,Single,College,2.0,320.0,2000.0,13000.0


In [5]:
auto = pd.read_csv('../car_insurance_real.csv')
auto.head()

,Vehicle Class,Coverage,Renew Offer Type,EmploymentStatus,Marital Status,Education,Number of Policies,Monthly Premium Auto,Total Claim Amount,Income
0,Two-Door Car,Basic,Offer1,Unemployed,Single,Bachelor,2.0,68.0,489.600000,0.0
1,Four-Door Car,Basic,Offer1,Employed,Married,College,1.0,73.0,59.861963,67763.0
2,Two-Door Car,Basic,Offer3,Unemployed,Single,College,1.0,65.0,468.000000,0.0
3,Four-Door Car,Basic,Offer1,Unemployed,Single,High School or Below,3.0,70.0,336.000000,0.0
4,Four-Door Car,Basic,Offer3,Unemployed,Married,Doctor,9.0,63.0,272.843439,0.0


In [6]:
# auto_filter = auto.loc[:, new_test_data.columns]
# auto_filter

In [7]:
# df = pd.read_csv('data_customer_lifetime_value.csv')
# df

In [8]:
# #merge two dataFrames and add indicator column
# all_df = pd.merge(df, auto_filter, on=list(auto_filter.columns), how='left', indicator='exists')

# #add column to show if each row in first DataFrame exists in second
# # all_df['exists'] = np.where(all_df.exists == 'both', 'exists', 'not exists')

# #view updated DataFrame
# # all_df[all_df['exists'] == 'both']
# all_df

In [9]:
# # new outlier test

# y_pred_out = random_search_xgb.best_estimator_.predict(life_out.drop(columns='Customer Lifetime Value'))
# print(adjusted_r2_score(life_out['Customer Lifetime Value'], y_pred_out, life_out.drop(columns='Customer Lifetime Value')))
# print(mean_absolute_percentage_error(life_out['Customer Lifetime Value'], y_pred_out))

# Variable Need for Google Cloud (or You could just upload all of them to Github)

In [ ]:
#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [12]:
## set up authentication using services account 
import os
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/sa-development.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/dti-ds-31329ac0651d.json"

project_id = 'dti-ds'
dataset_id = 'difgam_dataset_035'
table_id = 'carclv_dataset_difgam'
region = 'us-central1'
bucket_name = 'difgam_gcs_035'
blob_name = 'data/car_insurance_real.csv'

model_name = 'best_model_clv_xgb_20240715123616.sav'

## Upload to Google Cloud Storage

In [ ]:
# Upload the model to Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.upload_from_filename(f'{model_name}')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

## Retrieve from Google Cloud Storage

In [ ]:
# Retrieve the model from Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.download_to_filename('final_model_xgb.sav')

    print ("Read model succeeded")
except:
    raise TypeError("An exception occurred")

In [ ]:
# Retrieve the dataset from Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob(blob_name)
    blob_model.download_to_filename('car_insurance_real.csv')

    print ("Read Data succeeded")
except:
    raise TypeError("An exception occurred")

# Sent to BigQuery (Datset Creation)

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set dataset_id to the ID of the dataset to create.
# dataset_id = "{}.your_dataset".format(client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

# Load from Bigquery

In [ ]:
#load data from BQ
## using bigquery client 
# client = bigquery.Client(credentials=credentials,project=project_id)
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df = query_job.result().to_dataframe()

#cleansing 
# result = df.drop(['ID'], axis = 1)
result.head()

# Model Prediction

## Local Prediction (Please do not run this part on the cloud!!)

In [14]:
# local test
with open(f'../models/{model_name}', 'rb') as f:
    loaded_model = pickle.load(f)
loaded_model

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('nominal_preprocess',
                                                  Pipeline(steps=[('imputer_mode',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent')),
                                                                  ('onehot_encoding',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'int'>))]),
                                                  ['Vehicle Class',
                                                   'EmploymentStatus',
                                                   'Marital Status']),
                                                 ('ordinal_preprocess',
                                                  Pipeline(steps=[('imputer_mod...
                              feature_types=None, gamma=10.0, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.032421681936869715, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=3, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=256, n_jobs=-1,
                              num_parallel_tree=None, random_state=233, ...))])

In [15]:
loaded_model.predict(new_test_data)

array([ 5625.1167, 19218.45  ], dtype=float32)

In [16]:
loaded_model.predict(auto)

array([ 9868.214 ,  2943.0862,  2085.693 ,  5032.478 ,  4819.431 ,
        4983.1313,  5515.4155, 17676.86  ,  4225.032 ,  2922.464 ,
        8935.958 ,  4359.229 ,  6564.865 ,  4595.8994,  9805.145 ,
        5158.476 ,  4752.2207, 10537.784 ,  2395.616 ,  6272.6846,
        5419.4277,  3384.0422,  4275.4424,  7035.175 ,  5286.835 ,
        9521.179 ,  9876.644 ,  2501.6772,  4668.757 ,  2151.7637,
        7977.7393,  3640.1125, 10077.232 , 12729.727 ,  4852.403 ,
       16088.007 ,  2503.9336,  7117.071 ,  4463.9937,  5610.271 ,
        7994.4395, 12375.163 ,  5530.5356,  5847.036 ,  2697.607 ,
        9485.622 ,  3381.3801,  2709.1924, 13116.343 ,  6925.1333,
        2377.551 ,  2421.274 ,  6130.78  ,  5111.1064,  4268.6465,
        5254.448 ,  9192.791 , 12341.526 ,  8075.345 , 10770.437 ,
        5544.1206,  2355.1667,  3080.936 ,  5168.1855, 10321.045 ,
       12377.279 ,  6414.6562,  3387.1682,  5123.172 ,  5972.45  ,
        2847.5027,  4005.5542,  3261.068 ,  7979.071 ,  8134.1

## Cloud Data Prediction

In [22]:
# y_pred = loaded_model.predict(auto_filter)

In [17]:
# from sklearn.metrics import (
#     mean_absolute_error, mean_squared_error, r2_score,
#     mean_absolute_percentage_error, make_scorer,
#     root_mean_squared_error, PredictionErrorDisplay
# )

# mean_absolute_percentage_error(auto['Customer Lifetime Value'], y_pred), r2_score(auto['Customer Lifetime Value'], y_pred)

In [ ]:
# Load the model
with open('final_model_xgb.sav', 'rb') as f:
    loaded_model = pickle.load(f)

loaded_model.predict(new_test_data)

# loaded_model.predict([
#     [50_000, 1, 3, 1, 23, 131895, 132154, 129237],
#     [50_000, 0, 0, 1, 33, 1762, 881, 24842]
# ])

In [26]:
# loaded_model.predict(X_test.iloc[3:13])
loaded_model.predict(auto)

array([ 5625.1167, 19218.45  ], dtype=float32)

---
<center>Thank You</center>

---